In [1]:
# Quantum
import pennylane as qml

# Numericlal
import math
import numpy as np
import sympy as sp

# Torch
import torch
import torch.nn as nn
from torch import optim
from torch.nn import MSELoss
from torch.utils.data import TensorDataset, DataLoader, random_split

# Trainer
from trainer import trainer

# Plot
import matplotlib.pyplot as plt

%matplotlib inline


---

### Lemma 1

In [ ]:
class Lemma1:
    '''
        QAF 논문 Lemma 1을 구현하기 위한 class
    '''
    def __init__(self, w, x, b, output_num):
        '''
            w(matrix, tensor) : tensor for weights [row0 : w0, row1 : w1, ...], (output_num x input_num)
            x(tensor) : tensor for x(input, list)
            b(tensor) : tensor for bias_list
            output_num(int) : number of output perceptrons (same as len(b))
        '''
        # list(tensor) Initiallize
        self.w = w
        self.x = x
        self.b = b
        self.output_num = output_num

        # lengths Calculate
        self.N_in = len(x)
        self.N = int(2 ** (np.ceil(np.log2(self.N_in + 3))))
        self.n = int(np.log2(self.N))
        self.l = np.ceil(np.log2(self.output_num))

        # A_x, A_wb Calculate
        self.A_x  = torch.sqrt(torch.tensor(self.N_in) - (self.x @ self.x))
        self.A_wb = torch.sqrt(torch.tensor(self.N_in) + 1 - (self.w @ self.w + self.b * self.b))

        # v_x, v_wb
        self.v_x  = torch.cat(( self.A_x.unsqueeze(0)  ,self.x , torch.tensor([1.0]) ,torch.tensor([0.0] * (self.N - self.N_in - 2))))
        self.v_wb = torch.cat((torch.tensor([0.0]) ,self.w  ,self.b ,torch.tensor([0.0] * (self.N - self.N_in - 3)) , self.A_wb.unsqueeze(0)))

        # device initiallize
        # n : num_qubit
        self.dev = qml.device("default.qubit", wires=self.l+self.n)

    def norm(self, vec):
        return np.linalg.norm(vec)

    def u(self, vec):
        qml.AmplitudeEmbedding(vec, wires=range(self.n), normalize=True)


    def k_to_control_wire_list(self,k):
        return [ int(i) for i in bin(k)[2:]]
    
    def lemma1(self, chk=False):
        @qml.qnode(device=self.dev ,interface="torch")
        def inner_lemma1():

            for i in range(self.l):
                qml.Hadamard(wires=i)
            # U_x, i.e. small_u(v_x)
            qml.ctrl( self.u , )

            # U_wb, i.e. small_u(v_wb)^†, Pauli-X for each qubit
            qml.adjoint(self.u)(self.v_wb)
            for i in range(self.n):
                qml.PauliX(wires=i)

            return qml.state()
        if chk:
            print(qml.draw_mpl(inner_lemma1)())
        return inner_lemma1()[-1]

In [7]:
import numpy as np

def k_to_control_wire_list(k):
    tmp = bin(k)[2:]
    return [ int(i) for i in tmp]
k_to_control_wire_list(19)


[1, 0, 0, 1, 1]

---

### Theorem 1, Corollary 1

In [3]:
class Theorem1:
    '''
        QAF 논문 Theorem 1을 구현하기 위한 class
    '''
    def __init__(self, w, x, b, d, l = 0):
        '''
            w(list)     : list for weight
            x(list)     : list for x(input)
            b(float)    : float value of bias
            d(int)      : degree for Taylor (polynomial) Expansion
        '''
        # list Initiallize
        self.w_list = w
        self.x_list = x
        self.b = b
        self.d = d
        self.l = l

        # vector Initiallize
        self.x = np.array(self.x_list)
        self.w = np.array(self.w_list)

        # lengths Calculate
        self.N_in = len(x)
        self.N = int(2 ** (np.ceil(np.log2(self.N_in + 3))))
        self.n = int(np.log2(self.N))

        # A_x, A_wb Calculate
        self.A_x  = np.sqrt(self.N_in - (self.x @ self.x))
        self.A_wb = np.sqrt(self.N_in + 1 - (self.w @ self.w + self.b * self.b))

        # v_x, v_wb
        self.v_x  = [self.A_x]  + self.x_list + [1.0]       + ([0.0] * (self.N - self.N_in - 2))
        self.v_wb = [0.0]       + self.w_list + [self.b]    + ([0.0] * (self.N - self.N_in - 3)) + [self.A_wb]

        # device initiallize
        # n : num_qubit
        self.dev = qml.device("default.qubit", wires=(self.n + self.d + self.l))

    def U_z(self):
        qml.AmplitudeEmbedding(self.v_x, wires=range(self.n), normalize=True)
        qml.adjoint(qml.AmplitudeEmbedding)(self.v_wb, wires=range(self.n), normalize=True)
        for i in range(self.n):
            qml.PauliX(wires=i)

    def V(self, m):
        a_m = self.n + m

        # Controlled-Hadamard for (control, target) : (q register, a_m)
        qml.ctrl(qml.Hadamard, control=range(self.n))(wires=a_m)

        # CNOT for (control, target) : (a_m, q register)
        for i in range(self.n):
            qml.CNOT(wires=[a_m, i])

        # Controlled-U(x, w, b) for : (a_m, q register)
        qml.ctrl(self.U_z, control=a_m)()

    def outer_theorem1(self):
        '''
            Pauli-X 후 S_V(V_0, V_1, ... , V_{d-1})
        '''
        # Pauli-X for each qubit of q register
        for i in range(self.n):
            qml.PauliX(wires=i)

        # V_m for each m in (0, ... , d - 1)
        for m in range(self.d):
            self.V(m)

    def theorem1(self, chk=False):
        '''
            Theorem 1의 state |psi_z^d> 를 생성해서, state vector를 return
        '''
        @qml.qnode(device=self.dev)
        def inner_theorem1():
            self.outer_theorem1()
            return qml.state()

        if chk:
            print(qml.draw_mpl(inner_theorem1)())
        return inner_theorem1()

    def corollary1(self, k):
        '''
            k를 input으로 받아, |N-1>(q)|2^k - 1>(a) state에 대한 amplitude를 return
        '''
        result = self.theorem1()
        bitstring = ('1' * self.n) + ('0' * (self.d - k)) + ('1' * k)
        index = int(bitstring, 2)
        return result[index]

---

### Theorem 2

In [4]:
class Theorem2(Theorem1):
    def __init__(self, w, x, b, d, f, l = 0):
        '''
            w, x, b, d : same as Thm1
            f(str) : name of function which will be sympified
        '''

        # print('Before Thm1 Init')
        super().__init__(w, x, b, d, l=l)
        # print('After Thm1 Init')
        self.f = sp.sympify(f) # target f
        self.k = -1     # target f의 taylor expansion 중 계수가 0이 아닌 최저 차수
        self.C_d = 1    # f_d(z)와 실제 f(z)의 factoring constant
        self.theta_list = self.make_theta_list()


    def taylor_series_coefficients(self):
        # 심볼 정의
        z = sp.symbols('z')
        point = 0.0

        # 테일러 전개 계산
        taylor_expansion = sp.series(self.f, z, point, self.d + 1).removeO()

        # 계수 추출
        coeffs = [taylor_expansion.coeff(z, i) for i in range(self.d + 1)]
        return coeffs

    def make_theta_list(self):
        # Get Coeffs of Taylor (polynomial) Expansion of target f
        coefficients = self.taylor_series_coefficients()

        # Update k (target f의 taylor expansion 중 계수가 0이 아닌 최저 차수)
        for i in range(0, self.d + 1):
            if coefficients[i] != 0:
                self.k = i
                break

        theta_list = [0 for _ in range(self.d)]
        # target f = 0
        if self.k < 0:
            return theta_list

        # a_i = 0 for all i < k
        for i in range(self.k):
            theta_list[i] = -1 * math.pi / 2

        # a_k Initiallization
        theta_list[self.k] = math.atan(-1 * coefficients[self.k + 1] / coefficients[self.k])

        A = 1
        for i in range(self.k + 1, len(coefficients) - 1):
            A *= math.cos(theta_list[i - 1])
            theta_list[i] = math.atan(-1 * coefficients[i + 1] / coefficients[self.k] * A)

        # C_d update
        C_d = coefficients[self.k] # a_k
        for i in range(self.k, self.d):
            C_d /= math.cos(theta_list[i]) # 1/cos(theta)
        self.C_d = torch.tensor([float(C_d)])

        return theta_list

    def U(self, d):
        '''
            U_d Unitary for S_U iteration
        '''
        theta_list = self.theta_list # make_theta_list()
        for k in range(1, d):
            a_0 = self.n
            a_k = a_0 + k
            qml.ctrl(qml.RY, control=a_k, control_values=0)(2 * theta_list[k - 1], wires=a_0)
            qml.CNOT(wires=[a_0, a_k])

    def outer_theorem2(self):
        '''
            S_U := d register에는 U_d, q register에는 Pauli-X for each qubit
        '''
        # Pauli-X for each qubit of q register
        # and S_V
        self.outer_theorem1()

        qml.Barrier()

        # U_d
        self.U(self.d)

        qml.Barrier()

        # Pauli-X for each qubit of q register
        for i in range(self.n):
            qml.PauliX(wires=i)

    def theorem2(self, chk=False):
        @qml.qnode(device=self.dev, interface="torch")
        def inner_theorem2():
            self.outer_theorem2()
            return qml.state()
        if chk:
            print(qml.draw_mpl(self.outer_theorem2)())
        return inner_theorem2()

    def corollary2(self, k):
        '''
            as same as col 1,
            k를 input으로 받아, |N-1>(q)|2^k - 1>(a) state에 대한 amplitude를 return
        '''
        result = self.theorem2()
        bitstring = ('0' * self.n) + ('0' * (self.d - k)) + ('1' * k)
        index = int(bitstring, 2)
        return result[index] * self.C_d # * C_d


---

### Amplitude Estimation

In [5]:
class qaf_layer(Theorem2):
    def __init__(self, w, x, b, d, f):
        '''
            w, x, b, d : same as Thm1
            f(str) : name of function which will be sympified
        '''
        # print('Before Thm2 Init')
        super().__init__(w, x, b, d, f, l = 1)
        # print('After Thm2 Init')

        self.f = sp.sympify(f) # target f
        self.k = -1 # target f의 taylor expansion 중 계수가 0이 아닌 최저 차수
        self.theta_list = self.make_theta_list()

    def outer_controlled_theorem2(self):
        l_0 = self.n + self.d # index of l register

        qml.Hadamard(wires=l_0) # Hadamard for l register

        qml.Barrier()

        qml.ctrl(self.outer_theorem2, control=l_0)()

        qml.Barrier()

        qml.Hadamard(wires=l_0) # Hadamard for l register

    def controlled_theorem2(self):
        @qml.qnode(device=self.dev, interface="torch")
        def inner_controlled_theorem2():
            self.outer_controlled_theorem2()
            return qml.probs()
        return inner_controlled_theorem2()

    def amplitude_estimation(self):
        result = self.controlled_theorem2()[0] # get P_0
        factorized_inner_product = torch.sqrt(4 * result) - 1
        inner_product = factorized_inner_product * (2 ** (self.d / 2))
        return inner_product

    def activation_estimation(self):
        result = self.amplitude_estimation()
        return result * self.C_d


---

### QAF Model

In [9]:
# list Initiallize
x_list = [0.1, -0.2, -0.3, -0.4]
w_list = [-0.8, 0.7, 0.9, 0.3]
b = 0.5
d = 5
f = 'sin(z)'

args = [x_list, w_list, b, d, f]

N_in = len(x_list)
z = (np.array(w_list) @ np.array(x_list) + b) / (N_in + 1)

target_w = torch.tensor([-0.8, 0.7, 0.9, 0.3])
target_b = torch.tensor([0.5])
# 'f(wx+b) -> f(target_w * x + target_b)'

In [11]:
test1 = qaf_layer(w=w_list, x=x_list, b=b, d=d, f=f)
test1.activation_estimation(), math.sin(z)

(tensor([-0.0220]), -0.021998225376279792)